In [45]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
import psycopg2
from psycopg2 import sql

In [5]:
db_params={'dbname':'suraj',
                      'user':'postgres',
                      'password':'Tranzmeo1@#',
                      'host':'localhost',
                      'port':'5432'
}

In [ ]:
conn=psycopg2.connect(**db_params)
cursor=conn.cursor()
select_query="select * from transposed_df;"
df=pd.read_sql(select_query,conn)
df

In [34]:
df=df.drop('Unnamed: 0',axis=1)

In [35]:
X=df.iloc[:,:100]
y=df.iloc[:,-1]

In [36]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X=scaler.fit_transform(X)

In [37]:
from sklearn.preprocessing import LabelEncoder
en=LabelEncoder()
y=en.fit_transform(y)
y=to_categorical(y)

In [38]:
X=X.reshape(X.shape[0],X.shape[1],1)

In [39]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1) 

In [40]:
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y.shape[1], activation='softmax') 
])

In [41]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [42]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
65/65 [==============================] - 2s 14ms/step - loss: 0.8072 - accuracy: 0.6660 - val_loss: 0.6842 - val_accuracy: 0.7476
Epoch 2/20
65/65 [==============================] - 1s 11ms/step - loss: 0.6794 - accuracy: 0.7583 - val_loss: 0.6238 - val_accuracy: 0.7631
Epoch 3/20
65/65 [==============================] - 1s 11ms/step - loss: 0.6455 - accuracy: 0.7660 - val_loss: 0.5859 - val_accuracy: 0.8097
Epoch 4/20
65/65 [==============================] - 1s 11ms/step - loss: 0.6270 - accuracy: 0.7718 - val_loss: 0.5886 - val_accuracy: 0.7883
Epoch 5/20
65/65 [==============================] - 1s 11ms/step - loss: 0.6241 - accuracy: 0.7738 - val_loss: 0.5620 - val_accuracy: 0.8078
Epoch 6/20
65/65 [==============================] - 1s 11ms/step - loss: 0.6260 - accuracy: 0.7728 - val_loss: 0.5690 - val_accuracy: 0.8117
Epoch 7/20
65/65 [==============================] - 1s 11ms/step - loss: 0.6174 - accuracy: 0.7762 - val_loss: 0.5773 - val_accuracy: 0.8117
Epoch 8/20
65

In [43]:
loss,accuracy=model.evaluate(X_test,y_test)

17/17 [==============================] - 0s 3ms/step - loss: 0.5588 - accuracy: 0.8039


In [44]:
print(accuracy*100)

80.38834929466248
